# Libraries

In [ ]:
!pip install -q dwave-ocean-sdk

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
from google.colab import files
from google.colab import userdata
import pickle
import os
import shutil
from tqdm import tqdm

In [ ]:
from collections import defaultdict
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
import dwave.inspector as inspector
from dimod import ConstrainedQuadraticModel, CQM, SampleSet
from dwave.system import LeapHybridCQMSampler
from dimod.vartypes import Vartype
from dimod import Binary, quicksum

# DWAVE Initializing

In [ ]:
endpoint = 'https://cloud.dwavesys.com/sapi'
token = userdata.get('dwave_leap')

# QSAP Data Class

In [ ]:
class GQAPInstance:
    def __init__(self, filename):
        self.filename = filename
        self.instance_name = filename.split('.')[0]
        self.n = 0  # number of facilities
        self.m = 0  # number of sites
        self.w = 0  # capacity per site
        self.T = []  # Traffic matrix (n x n)
        self.D = []  # Distance matrix (m x m)
        self.A = []  # Assignment cost matrix (n x m)
        self.read_instance()

    def read_instance(self):
        with open(self.filename, 'r') as file:
            lines = file.readlines()

        # Read first line: n, m, w
        self.n, self.m, self.w = map(int, lines[0].strip().split())

        # Read Traffic matrix T (n x n)
        current_line = 1
        self.T = []
        for i in range(self.n):
            self.T.append(list(map(int, lines[current_line + i].strip().split())))
        current_line += self.n  # Move to next section after reading T

        # Read Distance matrix D (m x m)
        self.D = []
        for i in range(self.m):
            self.D.append(list(map(int, lines[current_line + i].strip().split())))
        current_line += self.m  # Move to next section after reading D

        # Read Assignment cost matrix A (n x m)
        self.A = []
        for i in range(self.n):
            self.A.append(list(map(int, lines[current_line + i].strip().split())))

        self.A = np.array(self.A)
        self.T = np.array(self.T)
        self.D = np.array(self.D)

    def display(self):
        print(f"Number of facilities (n): {self.n}")
        print(f"Number of sites (m): {self.m}")
        print(f"Capacity per site (w): {self.w}")
        print(f"\nTraffic Matrix (T), size {self.T.shape}:")
        for row in self.T:
            print(row)
        print(f"\nDistance Matrix (D), size {self.D.shape}:")
        for row in self.D:
            print(row)
        print(f"\nAssignment Cost Matrix (A), size {self.A.shape}:")
        for row in self.A:
            print(row)

    def write_matrix_log_format(self, matrix_name,output_filename, verbose = False):
        """
        Writes a matrix in log format to a file.
        matrix_name: 'T', 'D', or 'A'
        output_filename: name of the output file
        """
        if matrix_name == 'T':
            matrix = self.T
            rows, cols = self.n, self.n
        elif matrix_name == 'D':
            matrix = self.D
            rows, cols = self.m, self.m
        elif matrix_name == 'A':
            matrix = self.A
            rows, cols = self.n, self.m
        else:
            raise ValueError("Invalid matrix name. Use 'T', 'D', or 'A'.")

        # output_filename = f"{self.n}_{self.m}_{self.w}_{matrix_name}.txt"
        with open(output_filename, 'w') as file:
            for i in range(rows):
                for j in range(cols):
                    # Write log format (1-based indices)
                    file.write(f"{i+1} {j+1} {matrix[i][j]}\n")
        if verbose:
          print(f"Matrix {matrix_name} written to {output_filename} in log format.")

    def generate_all_output_logs(self,output_folder,verbose = False):
        os.makedirs(output_folder, exist_ok=True)
        for matrix_name in ['T', 'D', 'A']:
            output_filename = os.path.join(output_folder, f"{self.instance_name}_{self.n}_{self.m}_{self.w}_{matrix_name}_log.txt")
            self.write_matrix_log_format(matrix_name,output_filename, verbose)

# Import data

In [ ]:
instance = GQAPInstance('50-10-95.dat')

In [ ]:
instance.display()

# Creating Model

In [ ]:
N = range(instance.n)
M = range(instance.m)

In [ ]:
# create empty model
cqm = ConstrainedQuadraticModel()

In [ ]:
# create variables
x = {(i, k): Binary(f'x{i}_{k}') for i in N for k in M}

In [ ]:
total_iterations = (instance.n**2)*(instance.m**2)

In [ ]:
# create linear term of the objective function
linear_term = quicksum( x[i,k]*instance.A[i,k] for i in N \
                                for k in M)

quadratic_terms = 0

with tqdm(total=total_iterations, desc="Progress", unit="iteration") as pbar:
  for i in N:
    for j in N:
      for l in M:
        for k in M:
          quadratic_terms += x[i,k]*x[j,l]*instance.T[i,j]*instance.D[k,l]*instance.w
          pbar.update(1)

objective = linear_term + quadratic_terms

cqm.set_objective(objective)

In [ ]:
# # create linear term of the objective function
# linear_term = quicksum( x[i,k]*instance.A[i,k] for i in N \
#                                 for k in M)

# # create quadratic term of the objective function
# quadratic_term = quicksum( x[i,k]*x[j,l]*instance.T[i,j]*instance.D[k,l] \
#                                 for i in N \
#                                 for k in M \
#                                 for j in N \
#                                 for l in M)

# objective = linear_term + quadratic_term

# cqm.set_objective(objective)

In [ ]:
# A facility can only be assigned to only and only one site
for i in N:
  cqm.add_constraint( quicksum(x[i,k] for k in M) == 1 )

## Run the model

In [ ]:
counter = 1
while True:
  try:
    cqm_sampler = LeapHybridCQMSampler(endpoint=endpoint, token=token)
  except:
    if counter <= 5:
      print(f"{counter} -  Problem finding embedding trying it once again...")
      counter += 1
      continue
    else:
      raise Exception(f"Imposible to find an embedding after {counter} tries")
  break

In [ ]:
sampleset = cqm_sampler.sample_cqm(cqm,label = f'QSAP_{instance.instance_name}')

In [ ]:
feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)

In [ ]:
not_feasible_sampleset = sampleset.filter(lambda row: not row.is_feasible)

In [ ]:
best = feasible_sampleset.first

In [ ]:
print(f'Instance Name: {instance.instance_name}')
print(f'Best Objective Value = {best.energy:,.0f}')
print(f'Running Time = {(sampleset.info["run_time"]/1e6)/60:.5f} min')

In [ ]:
best_solution = best.sample

# Save sampleset in pickle file

In [ ]:
with open(f"{instance.instance_name}_dwave_sampleset.pkl", 'wb') as f:
  pickle.dump(sampleset, f)

In [ ]:
# !rm *